<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#imports" data-toc-modified-id="imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>imports</a></span></li><li><span><a href="#spectral-norm-from-scartch" data-toc-modified-id="spectral-norm-from-scartch-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>spectral norm from scartch</a></span></li><li><span><a href="#Testing" data-toc-modified-id="Testing-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Testing</a></span></li><li><span><a href="#Resources-used" data-toc-modified-id="Resources-used-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Resources used</a></span></li></ul></div>

# imports


In [1]:
import torch
from torch import nn
#from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch.nn.utils.parametrize as parametrize
torch.manual_seed(0)

In [2]:
torch.cuda.device_count() 
torch.cuda.is_available()

False

In [3]:
# crate random input x 
x= torch.rand((1,1,4,4))
# crate weights initialization method with random seed to generate same initialization values each call
#(only used for test my implemtion for spectral norm to compare different implementation )
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        torch.manual_seed(61)
        torch.nn.init.xavier_normal_(m.weight)
        

# spectral norm from scartch 

In [4]:

class sp(nn.Module):
    def forward(self, w,n_power_itter=1):
        norm = lambda a:(a/torch.norm(a))  # norm: normlization function  
        rows = w.shape[0]                # rows of the tensor 
        cols = w.view(rows, -1).shape[1] # columens of the tensor 
        w_vec=w.view(rows,-1)            # weights tensor to vector
        #print("rows : ",rows)               # prints for debug
        #print("cols : ",cols)
        u = norm(torch.randn((rows,1)))   # intalize u , v vectors 
        v = norm(torch.randn((cols,1)))
        #print("u: ",u.shape)                  # more prints for debug
        #print("v: ",v.shape)
        #print("w_vec: ",w_vec.shape)
        for _ in range(n_power_itter):               
            v =norm(torch.matmul(w_vec.T,u))
            u =norm(torch.matmul(w_vec,v))
        #print("u: ",u.shape)
        #print("v: ",v.shape)
        #print("w_vec: ",w_vec.shape)
        sigma_w =torch.matmul(u.T ,torch.matmul(w_vec,v))   
        #print(sigma)
        return w / sigma_w

# Testing
create 3 models with same wights each has one convolution layer :
1. model with only convolution layer.
2. model with spectral norm and convolution layer using spectral norm from pythorch .
3. model with spectral norm and convolution layer using my weights wrapper layer.

In [5]:
# create one cov layer without spectral norm to compare my implementation resultes with the frame work  
model = nn.Sequential(
                    nn.Conv2d(in_channels=1, out_channels=1, kernel_size =3,stride= 1,bias=False).apply(weights_init)
                        )
 
model = model.apply(weights_init)
#inspect the wieght and one forward pass 
print("----State_dict----")
print(model.state_dict())
print("..output..")
print(model(x))
print("----out----")

----State_dict----
OrderedDict([('0.weight', tensor([[[[-0.3380, -0.0287,  0.0131],
          [-0.1364,  0.1259,  0.2268],
          [-0.1098, -0.1079,  0.0697]]]]))])
..output..
tensor([[[[-0.1335, -0.1611],
          [-0.0193, -0.1820]]]], grad_fn=<ThnnConv2DBackward0>)
----out----


In [6]:
# create one layer with spectral norm to compare my implementation resultes with the frame work  
model_sp = nn.Sequential(
                nn.utils.spectral_norm(
                    nn.Conv2d(in_channels=1, out_channels=1, kernel_size =3,stride= 1,bias=False).apply(weights_init),
                           n_power_iterations=10)
                        )
 
model_sp = model_sp.apply(weights_init)
#inspect the wieght and one forward pass 
print("----State_dict----")
print(model_sp.state_dict())
print("..output..")
print(model_sp(x))
print("----out----")

----State_dict----
OrderedDict([('0.weight_orig', tensor([[[[-0.3380, -0.0287,  0.0131],
          [-0.1364,  0.1259,  0.2268],
          [-0.1098, -0.1079,  0.0697]]]])), ('0.weight_u', tensor([1.])), ('0.weight_v', tensor([ 0.0548,  0.3553, -0.4458,  0.3996,  0.0275, -0.0919,  0.4769, -0.1042,
         0.5146]))])
..output..
tensor([[[[-0.2786, -0.3360],
          [-0.0403, -0.3798]]]], grad_fn=<ThnnConv2DBackward0>)
----out----


In [7]:
model_my= nn.Sequential(
    parametrize.register_parametrization(
        nn.Conv2d(in_channels=1, out_channels=1, kernel_size =3,stride= 1,bias=False).apply(weights_init),"weight", sp()
    ))
#inspect the wieght and one forward pass to make sure it equals the layer with spectral_norm
print("----State_dict----")
print(model_my.state_dict())
print("..output..")
print(model_my(x))
print("----out----")

----State_dict----
OrderedDict([('0.parametrizations.weight.original', tensor([[[[-0.3380, -0.0287,  0.0131],
          [-0.1364,  0.1259,  0.2268],
          [-0.1098, -0.1079,  0.0697]]]]))])
..output..
tensor([[[[-0.2786, -0.3360],
          [-0.0403, -0.3798]]]], grad_fn=<ThnnConv2DBackward0>)
----out----


my weights wrapper layer gaves the same output as using spectral norm from pythorch.so i believe the code work  fine in gan model.

# Resources used

* [SPECTRAL NORMALIZATION FOR GENERATIVE ADVERSARIAL NETWORKS](https://arxiv.org/pdf/1802.05957v1.pdf)
* [github , pytorch-spectral-normalization-gan , by :christiancosgrove ](https://github.com/christiancosgrove/pytorch-spectral-normalization-gan/blob/12dcf945a6359301d63d1e0da3708cd0f0590b19/spectral_normalization.py#L14)
* [towardsdatascience ](https://towardsdatascience.com/lipschitz-continuity-and-spectral-normalization-b03b36066b0d)
* [medium](https://jonathan-hui.medium.com/gan-spectral-normalization-893b6a4e8f53)